In [ ]:
%matplotlib widget

# Fitting QENS data

Previously, some quasi-elastic neutron scattering (QENS) data has been [simulated](./../3-mcstas/QENS_from_function.ipynb) and [reduced](./../4-scipp/qens-reduction.ipynb) and can now be analysed with `easyscience`. 
Before the analysis can begin, it is necessary to load the experimental data and check that it looks reasonable. 
The data can be loaded with `np.loadtxt` as the data has been stored in a simple space-separated column file.

In [ ]:
import numpy as np
from load import load

q, i, di = load('../4-reduction/qens_energy_transfer_unknown_quasi_elastic_3_pulse.dat')

With the data read in, we can produce a quick plot. 

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.errorbar(q, i, di, fmt='.')
ax.set(xlabel='$\omega$/meV', ylabel='$I(\omega)$')
plt.show()

The QENS data covers a slightly broader range than we want to investigate.
Therefore, we will crop the data so that we are only using points between -0.06 and 0.06 meV. 

In [ ]:
sel = (q > -0.06) & (q < 0.06)
q, i, di = q[sel], i[sel], di[sel]

We now want to consider the mathematical model to be used in the analysis. 
QENS data is typically analysed by fitting a Lorentzian function to the data, which has the functional form

$$
I(\omega) = \frac{A\gamma}{\pi\big[(\omega - \omega_0) ^ 2 + \gamma ^ 2\big]},
$$ (lorentzian)

where $A$ is a scale factor, $\gamma$ is the half-width at half-maximum, and $\omega_0$ is the centre offset. 

## Exercise 1: implement a Lorentzian function

Write a function that implements Eqn. {eq}`lorentzian`, called `lorentzian`. 

**Solution:**

In [ ]:
# Insert your solution:


The instrument has a finite resolution, and to take this into account, the Lorentzian function must be convolved with this resolution. In our case, the resolution is a Gaussian distribution that is centred at zero with width, $\sigma$. 
In real experiments there are ways to measure $\sigma$, but for this project, we will simply model it. 

In [ ]:
from scipy.stats import norm

def intensity(omega):
    """
    The convolution of a Gaussian and the Lorentzian.
    
    :omega: the energy transfer values to calculate over.

    :return: intensity values.
    """
    gauss = norm(0, sigma.value).pdf(omega)
    gauss /= gauss.sum()
    return np.convolve(lorentzian(omega), gauss, 'same')

This means that there are a total of four parameters in our model. 

## Exercise 2: create fitting parameters

Create four `Parameter` objects, for $A$, $\gamma$, $\omega_0$, and $\sigma$.

Each should have an initial value and a uniform prior distribution, based on the values given in {numref}`qens-parameters`.

| Parameter | Initial Value | Min | Max |
| --- | --- | --- | --- |
| $A$ | 10 | 1 | 100 |
| $\gamma$ | 8.0 &times; 10<sup>-3</sup> | 1.0 &times; 10<sup>-4</sup> | 1.0 &times; 10<sup>-2</sup> |
| $\omega_0$ | 1.0 &times; 10<sup>-3</sup> | 0 | 2.0 &times; 10<sup>-3</sup> |
| $\sigma$ | 1.0 &times; 10<sup>-3</sup> | 1.0 &times; 10<sup>-5</sup> | 1.0 &times; 10<sup>-1</sup> |

**Solution:**

In [ ]:
# Insert your solution:


It is now possible to compare our model at the initial estimates to the simulated data. 

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(q, i, di, fmt='.')
ax.plot(q, intensity(q), '-k', zorder=10)
ax.set(xlabel='$\omega$/meV', ylabel='$I(\omega)$')
plt.show()

## Exercise 3: find maximum likelihood estimates

Using `easyscience`, obtain maximum likelihood estimates for the four parameters of the model from comparison with the data.

**Solution:**

In [ ]:
# Insert your solution:


We can then plot the model and the data together as before and print the values of the parameters along with their uncertainties. 

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(q, i, di, fmt='.')
ax.plot(q, intensity(q), '-k', zorder=10)
ax.set(xlabel='$\omega$/meV', ylabel='$I(\omega)$')
plt.show()

In [ ]:
A, gamma, omega_0, sigma